In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import requests
import yfinance as yf
import os
import json

# 🎯 Setores-alvo
SETORES_ALVO = {
    "Finance",
    "Utilities",
    "Communications",
    "Industrial Services"
}

# 🔁 Renomeações conhecidas
RENOMEACOES_TICKERS = {
    "TIMS3.SA": ["VIVT3.SA"],
    "PRIO3.SA": ["HRT3.SA"],
    "VBBR3.SA": ["BRDT3.SA"],
}

# 📦 Função principal
def gerar_dicionario_ativos(setores):
    print("📡 Buscando ativos no BRAPI...")
    try:
        response = requests.get("https://brapi.dev/api/quote/list")
        lista = response.json().get("stocks", [])
    except Exception as e:
        print(f"❌ Erro na requisição: {e}")
        return {}

    ativos = {}
    for item in lista:
        ticker = f"{item.get('stock')}.SA"
        setor = item.get("sector")
        tipo = item.get("type")

        if setor not in setores or tipo not in {"stock", "fund"} or ticker.endswith("F.SA"):
            continue

        try:
            info = yf.Ticker(ticker).info
            if info.get("country") != "Brazil":
                continue

            ativos[ticker] = {
                "empresa": info.get("longName", "N/A"),
                "setor": setor,
                "tipo": tipo,
                "logo": item.get("logo"),
                "aliases": RENOMEACOES_TICKERS.get(ticker, [])
            }
        except Exception:
            continue

    print(f"✅ {len(ativos)} ativos válidos encontrados.")
    return ativos

# 💾 Salvar JSON
def salvar_json(dados, caminho):
    os.makedirs(os.path.dirname(caminho), exist_ok=True)
    with open(caminho, "w", encoding="utf-8") as f:
        json.dump(dados, f, indent=4, ensure_ascii=False)
    print(f"💾 Dicionário salvo em: {caminho}")

# 🧠 Execução
if __name__ == "__main__":
    ativos = gerar_dicionario_ativos(SETORES_ALVO)
    caminho_json = r"E:\Github\Unicamp\Bussola-de-Valor\data\dicionario_ativos.json"
    salvar_json(ativos, caminho_json)


📡 Buscando ativos no BRAPI...
✅ 150 ativos válidos encontrados.
💾 Dicionário salvo em: E:\Github\Unicamp\Bussola-de-Valor\data\dicionario_ativos.json
